# 問1
  
## コード

### シミュレーション

#### 可読性を犠牲にして実効速度を優先したドローンクラス

In [ ]:
import numpy as np
import numpy.typing as npt


class Drone:
    _startTime = 0.0
    _change = 1

    def __init__(self, a: int, b: int, v: int):
        self._A = a
        self._B = b
        self._s = np.random.rand(2) * [a, b]
        self._d = np.random.rand(2) * [a, b]
        self.drone = self._s
        self._driveVelocity = v

    @property
    def change(self):
        return self._change

    def _droneMove(self, t: float):
        sdAngle = np.arctan2(self._d[1] - self._s[1], self._d[0] - self._s[0])
        dronevArr = self._driveVelocity * np.array([np.cos(sdAngle), np.sin(sdAngle)])
        sectionTime = t - self._startTime
        self.drone = self._s + dronevArr * sectionTime

    def _changeDestination(self):
        self._startTime += np.linalg.norm(self._s - self._d) / self._driveVelocity
        self._s = self._d
        self.drone = self._d
        self._d = np.random.rand(2) * [self._A, self._B]
        self._change += 1

    def advanceTime(self, t: float):
        if t < self._startTime:
            raise ValueError("Time cannot go backwards")

        while t - self._startTime > (
            np.linalg.norm(self._s - self._d) / self._driveVelocity
        ):
            self._changeDestination()

        self._droneMove(t)


#### シミュレーションクラス

In [ ]:
class Sim:
    _contacted = False
    _topCount = 0
    _buttomCount = 0

    def __init__(self, a, b, v, rng):
        self._n1 = Drone(a, b, v)
        self._n2 = Drone(a, b, v)
        self._V = v
        self._rang = rng

    def check_distace(self, distance):
        return self._rang > distance

    def counter(self, contact: bool):
        if contact:
            self._contacted = True
        else:
            if self._contacted:
                self._buttomCount += 1
            self._topCount += 1
            self._contacted = False

    def main(self):

        T = 2000000
        res = 1

        n1route = []
        n2route = []
        distances = []
        contacts = []

        for i in range(1, T):
            t = i / res
            self._n1.advanceTime(t)
            self._n2.advanceTime(t)
            n1route.append(self._n1.drone)
            n2route.append(self._n2.drone)
            if (i / T) * 100 % 10 == 0:
                print("rang:", self._rang, "progress:", (i / T) * 100, "%")

        distances = np.linalg.norm(np.array(n1route) - np.array(n2route), axis=1)
        contacts = distances < self._rang

        for contact in contacts:
            self.counter(contact)

        print(self._topCount / self._buttomCount)
        print(self._topCount)
        print(self._buttomCount)
        el = T * self._V * 2 / (self._n1.change + self._n2.change)
        return [self._topCount / self._buttomCount, el]


### 理論値

In [ ]:
import math
import numpy as np
from scipy.integrate import dblquad

def theoroticalEL(a, b):
    term1 = (a**3 / b**2 + b**3 / a**2 + math.sqrt(a**2 + b**2) * (3 - a**2 / b**2 - b**2 / a**2)) / 15
    term2 = (b**2 / a * math.acosh(math.sqrt(a**2 + b**2) / b) + a**2 / b * math.acosh(math.sqrt(a**2 + b**2) / a)) / 6
    E_L = term1 + term2
    return E_L

class Theorotical_ETc:
    

    def __init__(self, a, b, v, r):
        self._A = a
        self._B = b
        self._V = v
        self._R = r

    def _fDeltaX(self, x):
        return 12 * (self._A - x)**3 * (self._A**2 + 3*self._A*x + x**2) / (5 * self._A**6)


    def _fDeltaY(self, y):
        return 12 * (self._B - y)**3 * (self._B**2 + 3*self._B*y + y**2) / (5 * self._B**6)


    def _integrand(self, y, x):
        return self._fDeltaX(x) * self._fDeltaY(y)

    def main(self):
        PrRr = dblquad(self._integrand, 0, min(self._R, self._B), lambda y: 0, lambda y: min(np.sqrt(self._R**2 - y**2), self._A))[0]

        ETCc = (np.pi**2 * self._R) / (8 * self._V)

        # 式(12) を計算
        ETc = ETCc / PrRr

        return ETc    


### シミュレーションの実行

In [ ]:
a = 10000  # サービスエリアの横の長さ (m)
b = 10000  # サービスエリアの縦の長さ (m)
v = 10  # 端末の移動速度 (m/s)

results = []

for r in range(100, 1000, 100):
    ETcTheo = Theorotical_ETc(a, b, v, r)
    ElTheo = theoroticalEL(a, b)
    sim = Sim(a, b, v, r).main()    
    results.append([r, ETcTheo.main(), ElTheo, sim[0], sim[1]])


In [ ]:
import matplotlib.pyplot as plt
# グラフの描画
results = np.array(results)
plt.plot(results[:, 0], results[:, 1], label='E(Tc)理論値')
plt.plot(results[:, 0], results[:, 2], label='E(L)理論値')
plt.plot(results[:, 0], results[:, 3], label='E(Tc)シミュレーション')
plt.plot(results[:, 0], results[:, 4], label='E(L)シミュレーション')
plt.xlabel('R')
plt.ylabel('ETc')

## 結果
シミュレーション値と理論値に若干の誤差が出てしまったが、これは試行回数が足りないものによって発生していると考えられる。  
実際に試行回数を増やすと理論値にかなり近い値が得られることは確認済みである。
現在のコードも高速化を図ったが、さらなる工夫をすることは今後の課題としていきたい。